<br />
<br />

<center><h1><b>Lecture 15</b></h1></center>
<center><h1><b>Introduction to Concurrency and Parallelism</b></h1></center>

<center><h4>Time: M56 (13:20 ~ 15:10) and R2 (09:00 ~ 09:50)</h4></center>
<br />

<center><h2>Chieh-En Lee<sup>1</sup> (李杰恩) and Chung-Hao Tien<sup>2</sup> (田仲豪)</h2></center>

<center>
<h4>{<a href="mailto:celee@nycu.edu.tw">celee</a><sup>1</sup>, 
<a href="mailto:chtien@nycu.edu.tw">chtien</a><sup>2</sup>}@nycu.edu.tw</h4>
</center>

<center><h3><a href="https://dop.nycu.edu.tw/ch/index.html">Department of Photonics</a>, <a href="https://www.nycu.edu.tw/">NYCU</a></h3></center>

<br />
<center><h5><a href="https://github.com/bruce88617/nycudopcs">Introduction to Computer and Computer Science</a>, 2022 Fall</h5></center>


## Last time

- Searching algorithms  
- Sorting algorithms  
- Lambda  

## **Today**

<html>
<head>
</head>
<body>
<ul>
  <li><a href="#tag1">Concurrency vs. parallelism</a></li>
  <li><a href="#tag2">Multi-threading</a></li>
  <li><a href="#tag3">Multi-processing</a></li>
</ul>

</body>

<a id="tag1"></a>

## **Concurrency vs. parallelism**  

>
> Concurrency is about dealing with lots of things at once.  
> Parallelism is about doing lots of things at once.  
> 
>                                           - Rob Pike
> 

<img align="center" height=auto width=600px src="https://raw.githubusercontent.com/bruce88617/nycudopcs/main/Lectures/Lecture15/assets/fig1.png">

- - -

#### Practical example

<img align="center" height=auto width=700px src="https://raw.githubusercontent.com/bruce88617/nycudopcs/main/Lectures/Lecture15/assets/fig2.png">

- - -

<a id="tag2"></a>

## **Multi-threading**  

#### Python build-in module `threading` 

- Thread-based parallelism
- Suitable for heavy I/O task (e.g. reading & writing files)

In [6]:
import threading
import time

def job1(msg, t=1):
    print("Current thread:", threading.current_thread())
    print("Active threads:", threading.active_count())
    print("Print message: {} is working.".format(msg))
    time.sleep(t)
    print("{} is done.".format(msg))

def job2(msg, t=5):
    print("Current thread:", threading.current_thread())
    print("Active threads:", threading.active_count())
    print("Print message: {} is working.".format(msg))
    time.sleep(t)
    print("{} is done.".format(msg))

#### Example 1: create 2 threads by `Thread`

In [7]:
thread1 = threading.Thread(target=job1, name="T1", args=("Thread 1",))
thread2 = threading.Thread(target=job2, name="T2", args=("Thread 2",))

print("Current thread:", threading.current_thread())

print("="*50)
thread1.start()
thread2.start()
print("="*50)

print("Current thread:", threading.current_thread())
print("Active threads:", threading.active_count())


Current thread: <_MainThread(MainThread, started 17416)>
Current thread: <Thread(T1, started 8184)>
Active threads: 7
Print message: Thread 1 is working.
Current thread: <Thread(T2, started 17244)>
Active threads: 8
Print message: Thread 2 is working.
Current thread: <_MainThread(MainThread, started 17416)>
Active threads: 8


Thread 1 is done.
Thread 2 is done.


#### Example 2: `join`

-  Temporately stop the `MainThread` until all thread has finished.

In [8]:
thread1 = threading.Thread(target=job1, name="T1", args=("Thread 1",))
thread2 = threading.Thread(target=job2, name="T2", args=("Thread 2",))

print("Current thread:", threading.current_thread())

print("="*50)
thread1.start()
thread2.start()
thread1.join()
thread2.join()
print("="*50)

print("Current thread:", threading.current_thread())
print("Active threads:", threading.active_count())

Current thread: <_MainThread(MainThread, started 17416)>
Current thread: <Thread(T1, started 4976)>
Active threads: 7
Print message: Thread 1 is working.
Current thread: <Thread(T2, started 16676)>
Active threads: 8
Print message: Thread 2 is working.
Thread 1 is done.
Thread 2 is done.
Current thread: <_MainThread(MainThread, started 17416)>
Active threads: 6


#### Example 3: `daemon` thread

- Automatically stop the thread when `MainThread` terminates.
- There are bugs in `ipython kernel`.

In [10]:
thread1 = threading.Thread(target=job1, name="T1", args=("Thread 1",), daemon=True)
thread2 = threading.Thread(target=job2, name="T2", args=("Thread 2",), daemon=True)

print("Current thread:", threading.current_thread())

print("="*50)
thread1.start()
thread2.start()
print("="*50)

print("Current thread:", threading.current_thread())
print("Active threads:", threading.active_count())

Current thread: <_MainThread(MainThread, started 17416)>
Current thread: <Thread(T1, started daemon 17108)>
Active threads: 7
Print message: Thread 1 is working.
Current thread: <Thread(T2, started daemon 9968)>
Active threads: 8
Print message: Thread 2 is working.
Current thread: <_MainThread(MainThread, started 17416)>
Active threads: 8


Thread 1 is done.
Thread 2 is done.


In [55]:
class multithread1:
    def __init__(self, input_list):
        self.data = input_list.copy()

    def get_data(self):
        return self.data

    def job1(self, data, idx):
        print("Thread {} is starting.".format(idx))
        for j in range(len(data)):
            self.data[idx][j] = data[j]**2
            time.sleep(1)

    def run(self):

        all_thread = []

        for i in range(len(self.data)):
            thread = threading.Thread(target=self.job1, name="T{}".format(i), args=(self.data[i],i))
            
            thread.start()
            all_thread.append(thread)    
        
        for thread in all_thread:
            thread.join()


In [56]:
N = 3

arr1 = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]

test1 = multithread1(arr1)
test1.run()
print(test1.get_data())

Thread 1 is starting.
Thread 1 is starting.
Thread 1 is starting.
Thread 1 is starting.
[[1, 2, 3], [100, 4, 9], [7, 8, 9], [10, 11, 12]]


<a id="tag3"></a>

## **Multi-processing**  